# Анализ финансовых рядов.
Самой популярной криптовалютой является Bitcoin, стоимость которого уже превысила $15k. Однако, в мире существуют еще несколько тысяч альтернативных криптовалют, большая часть из которых ничего особенного из себя не представляют и даже динамика их цен совпадает с флагманским биткоином. 
- Задача найти криптовалюты, которые не будут попадать в кластер биткоина на разных временных промежутках

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.autonotebook import tqdm
import random

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import cryptocompare

/var/folders/84/q1qfkkn11dn0bkqq_3fg56040000gn/T/ipykernel_52078/1836415664.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/Users/alsukurmakaeva/miniconda3/envs/dqual_alpha_nlp/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
name_tikers_all = cryptocompare.get_coin_list(format=True)

In [3]:
len(name_tikers_all)

8410

In [4]:
name_tikers = random.sample(name_tikers_all, 300)

In [5]:
'BTC' in name_tikers

False

In [6]:
name_tikers.append('BTC')

Скачайем цены закрытия для имеющихся криптовалют по трем разным временным промежуткам:
- За последние 30 дней с дневной частотой
- За последние 3 дня с часовой частотой 
- За последний час с минутной частотой 

In [7]:
data_dayFreq = pd.DataFrame()
data_hourFreq = pd.DataFrame()
data_minuteFreq = pd.DataFrame()

first_l = [np.nan for i in range(3*24)]

for tiker in name_tikers:
    data_dayFreq[tiker] = first_l
    data_hourFreq[tiker] = first_l
    data_minuteFreq[tiker] = first_l

/var/folders/84/q1qfkkn11dn0bkqq_3fg56040000gn/T/ipykernel_52078/3965546325.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_dayFreq[tiker] = first_l
/var/folders/84/q1qfkkn11dn0bkqq_3fg56040000gn/T/ipykernel_52078/3965546325.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_hourFreq[tiker] = first_l
/var/folders/84/q1qfkkn11dn0bkqq_3fg56040000gn/T/ipykernel_52078/3965546325.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

In [8]:
bad_tikers_dayFreq = []
for tiker in tqdm(name_tikers):
    # print(tiker)
    try:
        for i in range(30):
            time.sleep(0.500)
            data_dayFreq[tiker][i] = cryptocompare.get_historical_price_day(tiker, currency='USD', limit=30)[i]['close']
    except Exception:
        bad_tikers_dayFreq.append(tiker) 
        continue

  0%|▏                                          | 1/301 [00:00<04:20,  1.15it/s]

[ERROR] CCCAGG market does not exist for this coin pair (TASH-USD)


  1%|▌                                          | 4/301 [00:49<58:03, 11.73s/it]

[ERROR] CCCAGG market does not exist for this coin pair (ZEP-USD)


  2%|▊                                          | 6/301 [01:14<53:47, 10.94s/it]

...

[ERROR] You are over your rate limit please upgrade your account!


100%|████████████████████████████████████████▊| 300/301 [32:14<00:00,  1.26it/s]

[ERROR] You are over your rate limit please upgrade your account!


100%|█████████████████████████████████████████| 301/301 [32:15<00:00,  6.43s/it]

[ERROR] You are over your rate limit please upgrade your account!


In [9]:
data_dayFreq

,TASH,SCC,BKK,ZEP,CELO,TFI,OOKI,KIBA,DINU,MDM,...,EXCC,SHPING,YCE,IDHUB,OTO,XSI,FST,XETH,SANA,BTC
0,NaN,0.000006,0.1294,NaN,0.627,NaN,0.003455,NaN,NaN,1.445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.000006,0.1327,NaN,0.635,NaN,0.003531,NaN,NaN,1.473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.000007,0.1419,NaN,0.685,NaN,0.003724,NaN,NaN,1.585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.000007,0.1391,NaN,0.685,NaN,0.003876,NaN,NaN,1.593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.000007,0.1392,NaN,0.685,NaN,0.003801,NaN,NaN,1.588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
bad_tikers_hourFreq = []
for tiker in tqdm(name_tikers):
    try:
        for i in range(3*24):
            time.sleep(0.100)
            data_hourFreq[tiker][i] = cryptocompare.get_historical_price_hour(tiker, currency='USD', limit=3*24)[i]['close']
    except Exception:
        bad_tikers_hourFreq.append(tiker) 
        continue

  0%|▏                                          | 1/301 [00:00<01:40,  2.97it/s]

[ERROR] You are over your rate limit please upgrade your account!


  1%|▎                                          | 2/301 [00:00<01:40,  2.97it/s]

[ERROR] You are over your rate limit please upgrade your account!


  1%|▍                                          | 3/301 [00:01<01:51,  2.68it/s]

...

[ERROR] You are over your rate limit please upgrade your account!


 95%|███████████████████████████████████████  | 287/301 [01:52<00:05,  2.43it/s]

[ERROR] You are over your rate limit please upgrade your account!


 96%|███████████████████████████████████████▏ | 288/301 [01:53<00:07,  1.83it/s]

[ERROR] You are over your rate limit please upgrade your account!


In [ ]:
bad_tikers_hourFreq

In [ ]:
bad_tikers_minuteFreq = []
for tiker in tqdm(name_tikers):
    try:
        for i in range(60):
            time.sleep(0.100)
            data_minuteFreq[tiker][i] = cryptocompare.get_historical_price_minute(tiker, currency='USD', limit=60)[i]['close']
    except Exception:
        bad_tikers_minuteFreq.append(tiker) 
        continue           

In [ ]:
data_minuteFreq

Удалим незаполненные тикеры

In [ ]:
data_dayFreq

In [ ]:
data_minuteFreq

In [ ]:
bad_tikers_dayFreq

In [ ]:
msno.matrix(data_dayFreq)